# Import Libraries

In [ ]:
import pandas as pd
import sqlite3
from sqlite3 import Error
from IPython.display import display
# this library will let us interact with websites on the internet (like our API). 
# Named as such because we "request" data from these API endpoints
import requests
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create Synonym DB

In [ ]:
conn = None;
try:
    conn = sqlite3.connect('labs.db')
    print(sqlite3.version)
    cursor = conn.cursor()
    
    # Drop the tables if they already exist.
    cursor.execute("DROP TABLE IF EXISTS Patients")
    cursor.execute("DROP TABLE IF EXISTS Admissions")
    cursor.execute("DROP TABLE IF EXISTS D_icd_diagnoses")
    cursor.execute("DROP TABLE IF EXISTS Diagnoses_icd")
    cursor.execute("DROP TABLE IF EXISTS d_labitems")
    cursor.execute("DROP TABLE IF EXISTS labevents")
    cursor.execute("DROP TABLE IF EXISTS drugs")
 
    # Create the following six tables:
    # Patients, Admissions, D_icd_diagnoses, Diagnoses_icd, d_labitems, labevents
    patients_create ='''CREATE TABLE Patients (
      row_id int  NOT NULL,
      subject_id int  NOT NULL,
      gender varchar(5)  NOT NULL,
      dob datetime  NOT NULL,
      dod datetime  NOT NULL,
      dod_hosp datetime  NULL,
      dod_ssn datetime  NULL,
      expire_flag varchar(5)  NOT NULL,
      PRIMARY KEY (subject_id)
    );'''
    cursor.execute(patients_create)
    admissions_create ='''CREATE TABLE Admissions (
      row_id int  NOT NULL,
      subject_id int  NOT NULL,
      hadm_id int  NOT NULL,
      admittime datetime  NOT NULL,
      dischtime datetime  NOT NULL,
      deathtime datetime   NULL,
      admission_type varchar(50)  NOT NULL,
      admission_location varchar(50)  NOT NULL,
      discharge_location varchar(50)  NOT NULL,
      insurance varchar(255)  NOT NULL,
      language varchar(10)   NULL,
      religion varchar(50)   NULL,
      marital_status varchar(50)   NULL,
      ethnicity varchar(200)  NOT NULL,
      edregtime datetime   NULL,
      edouttime datetime   NULL,
      diagnosis varchar(300)  NOT NULL,
      hospital_expire_flag int  NOT NULL,
      has_chartevents_data int  NOT NULL,
      PRIMARY KEY (hadm_id),
      FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
    );'''
    cursor.execute(admissions_create)
    # create D_icd_diagnoses
    icds_create ='''CREATE TABLE D_icd_diagnoses (
    row_id int  NOT NULL,
    icd9_code varchar(10)  NOT NULL,
    short_title varchar(50)  NOT NULL,
    long_title varchar(255)  NOT NULL,
    PRIMARY KEY (icd9_code)
  );'''
    cursor.execute(icds_create)
    # create Diagnoses_icd table
    diagnoses_create ='''CREATE TABLE Diagnoses_icd (
      row_id int  NOT NULL,
      subject_id int  NOT NULL,
      hadm_id int  NOT NULL,
      seq_num int  NULL,
      icd9_code varchar(10)  NULL,
      PRIMARY KEY (row_id),
      FOREIGN KEY (subject_id) REFERENCES Patients(subject_id),
      FOREIGN KEY (hadm_id) REFERENCES Admissions(hadm_id),
      FOREIGN KEY (icd9_code) REFERENCES D_icd_diagnoses(icd9_code)
    );'''
    cursor.execute(diagnoses_create)
    # create d_labitems
    labitems_create ='''CREATE TABLE d_labitems (
      row_id int  NOT NULL,
      itemid int  NOT NULL,
      label  varchar(100)  NOT NULL,
      fluid  varchar(100)  NOT NULL,
      category varchar(100) NOT NULL,
      loinc_code varchar(100)  NULL,
      PRIMARY KEY (row_id)
    );'''
    cursor.execute(labitems_create)
    # create labevents
    labevents_create ='''CREATE TABLE labevents (
      row_id int  NOT NULL,
      subject_id int  NOT NULL,
      hadm_id int  NULL,
      itemid int  NOT NULL,
      charttime datetime  NULL,
      value varchar(200)  NULL,
      valuenum float  NULL,
      valueuom varchar(20)  NULL,
      flag varchar(20)  NULL,
      PRIMARY KEY (row_id),
      FOREIGN KEY (subject_id) REFERENCES Patients(subject_id),
      FOREIGN KEY (hadm_id) REFERENCES Admissions(hadm_id),
      FOREIGN KEY (itemid) REFERENCES d_labitems(item_id)
    );'''
    cursor.execute(labevents_create)
    # create drugs
    drugs_create ='''CREATE TABLE drugs (
      index int  NULL,
      SUBJECT_ID int  NULL,
      HADM_ID int  NULL,
      STARTDATE datetime  NULL,
      ENDDATE datetime  NULL,
      DRUG varchar(255)  NULL,
      QUANTITY varchar(20)  NULL,
      UNIT varchar(20)  NULL,
      FREQUENCY varchar(50)  NULL,
      ROUTE varchar(50)  NULL,
      CLASS varchar(20)  NULL,
      PRIMARY KEY (index)
    );'''
    cursor.execute(drugs_create)

    conn.commit()    
    print("Table(s) created successfully........")
    
except Error as e:
        print(e)



2.6.0
near "index": syntax error


In [ ]:
try:
    
    # IMPORTANT: Use these names for your DataFrames, otherwise we won't be able to check them! 
    # Patients, Admissions, D_icd_diagnoses, Diagnoses_icd, d_labitems, labevents
    patients = None
    admissions = None
    diagnoses = None
    icds = None
    labitems = None
    labevents = None

    # load the MIMIC III datasets into data frame objects 
    patients = pd.read_csv('/content/drive/My Drive/Labs/PATIENTS.csv')
    admissions = pd.read_csv('/content/drive/My Drive/Labs/ADMISSIONS.csv')
    diagnoses = pd.read_csv('/content/drive/My Drive/Labs/DIAGNOSES_ICD.csv')
    icds = pd.read_csv('/content/drive/My Drive/Labs/D_ICD_DIAGNOSES.csv')
    labitems = pd.read_csv('/content/drive/My Drive/Labs/D_LABITEMS.csv')
    labevents = pd.read_csv('/content/drive/My Drive/Labs/LABEVENTS.csv')
    labevents = labevents.dropna(subset=['VALUENUM'])
    drugs = pd.read_csv('/content/drive/My Drive/Labs/combined_drugs_prescriptions.csv').reset_index()
    drugs = drugs[drugs.CLASS=='opioids']

    # write the data to a sqlite table
    patients.to_sql('Patients', conn, if_exists='replace', index = False)
    admissions.to_sql('Admissions', conn, if_exists='replace', index = False)
    diagnoses.to_sql('Diagnoses_icd', conn, if_exists='replace', index = False)
    icds.to_sql('D_icd_diagnoses', conn, if_exists='replace', index = False)
    labitems.to_sql('d_labitems', conn, if_exists='replace', index = False)
    labevents.to_sql('labevents', conn, if_exists='replace', index = False)
    drugs.to_sql('drugs', conn, if_exists='replace', index = False)

    
    # Commit your changes in the database
    conn.commit()
    
except Error as e:
    print(e)
    


In [ ]:
try:
  cursor.execute("DROP TABLE IF EXISTS Patients_info")
  sql='''
  CREATE TABLE Patients_info AS
    SELECT *,
    CASE WHEN t.AGE >= 90 THEN 90 ELSE t.AGE END AS AGE2, 
    CASE WHEN DIAGNOSIS LIKE "%pneumonia" THEN 1 ELSE 0 END AS PNEUMONIA 
      FROM(
        SELECT p.ROW_ID, p.SUBJECT_ID, p.GENDER, p.DOB, p.EXPIRE_FLAG, a.ADMITTIME, a.DISCHTIME, a.DIAGNOSIS,
          Cast ((JulianDay(a.ADMITTIME) - JulianDay(p.DOB))/365 As Integer) as AGE,
          Cast ((JulianDay(a.DISCHTIME) - JulianDay(a.ADMITTIME)) As Integer) as ADMISSION_TIME
        FROM Patients as p LEFT JOIN Admissions as a
        ON p.SUBJECT_ID = a.SUBJECT_ID
      )t
  '''
  cursor.execute(sql)

  cursor.execute("DROP TABLE IF EXISTS Labs_info")
  sql='''
  CREATE TABLE Labs_info AS
    SELECT p.*, l.ITEMID, l.VALUENUM, l.VALUEUOM, l.FLAG
    FROM Patients_info as p 
    LEFT JOIN labevents as l ON p.SUBJECT_ID=l.SUBJECT_ID 
  '''
  cursor.execute(sql)

  conn.commit()    
  print("Table(s) created successfully........")
except Error as e:
        print(e)

Table(s) created successfully........


In [ ]:
cursor.execute("DROP TABLE IF EXISTS Statistics")
sql='''
  CREATE TABLE Statistics AS
    SELECT DISTINCT l.SUBJECT_ID, l.GENDER, l.DIAGNOSIS, l.AGE2, l.PNEUMONIA, l.ADMISSION_TIME, 
    l.VALUENUM, l.VALUEUOM, l.FLAG, d.LABEL, d.FLUID, d.CATEGORY
    FROM Labs_info as l LEFT JOIN d_labitems as d
    ON l.ITEMID = d.ITEMID 
    WHERE d.LABEL IN ('C-Reactive Protein', 'White Blood Cells', 'Red Blood Cells', 'Glucose')
  '''
cursor.execute(sql)
conn.commit() 

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Important_Statistics")
sql='''
  CREATE TABLE Important_Statistics AS
    SELECT SUBJECT_ID, AGE_GROUP, PNEUMONIA, GENDER, 
    CASE WHEN LABEL = 'White Blood Cells' THEN VALUENUM ELSE NULL END AS WHITE_BLOOD_CELLS,
    CASE WHEN LABEL = 'C-Reactive Protein' THEN VALUENUM ELSE NULL END AS `C_REACTIVE_PROTEIN`
    FROM(
      SELECT *, CASE WHEN AGE2>=65 THEN 1 ELSE 0 END AS AGE_GROUP
      FROM Statistics
      WHERE LABEL IN ('C-Reactive Protein', 'White Blood Cells')
    )t1
  '''
cursor.execute(sql)
conn.commit() 

In [ ]:
try:
    sql='''
    SELECT DISTINCT AGE_GROUP, PNEUMONIA,
    ROUND(AVG(WHITE_BLOOD_CELLS) OVER(PARTITION BY AGE_GROUP, PNEUMONIA),2) as `Avg(White Blood Cells Count(k/uL))`,
    ROUND(AVG(C_REACTIVE_PROTEIN) OVER(PARTITION BY AGE_GROUP, PNEUMONIA),2) as `Avg(C-Reactive Protein(mg/dL))`
    FROM Important_Statistics
    ORDER BY 1 DESC, 2 DESC
    '''
    syn = pd.read_sql(sql, conn)
    display (syn)
except Error as e:
    print(e)

,AGE_GROUP,PNEUMONIA,Avg(White Blood Cells Count(k/uL)),Avg(C-Reactive Protein(mg/dL))
0,1,1,12.30,65.67
1,1,0,11.38,59.90
2,0,1,10.70,57.11
3,0,0,10.58,47.06


In [ ]:
try:
    sql='''
    SELECT DISTINCT AGE_GROUP, PNEUMONIA, GENDER,
    ROUND(AVG(WHITE_BLOOD_CELLS) OVER(PARTITION BY AGE_GROUP, PNEUMONIA, GENDER),2) as `Avg(White Blood Cells Count(k/uL))`,
    ROUND(AVG(C_REACTIVE_PROTEIN) OVER(PARTITION BY AGE_GROUP, PNEUMONIA, GENDER),2) as `Avg(C-Reactive Protein(mg/dL))`
    FROM Important_Statistics
    WHERE AGE_GROUP = 1
    ORDER BY 1 DESC, 2 DESC
    '''
    syn2 = pd.read_sql(sql, conn)
    display (syn2)
except Error as e:
    print(e)

,AGE_GROUP,PNEUMONIA,GENDER,Avg(White Blood Cells Count(k/uL)),Avg(C-Reactive Protein(mg/dL))
0,1,1,F,12.23,57.57
1,1,1,M,12.37,74.59
2,1,0,F,11.39,54.96
3,1,0,M,11.37,64.38


In [ ]:
try:
    sql='''
    with tmp as (
        SELECT i.*, t.DRUG_NUM,
        CASE WHEN t.DRUG_NUM IS NULL THEN 0 ELSE 1 END AS OPIOIDS
        FROM Important_Statistics as i LEFT JOIN (
          SELECT SUBJECT_ID, COUNT(DRUG) as DRUG_NUM
          FROM drugs
          GROUP BY 1
        )t
        ON i.SUBJECT_ID = t.SUBJECT_ID
    )
    SELECT AGE_GROUP, OPIOIDS, CAST(SUM(PNEUMONIA)*100/COUNT(OPIOIDS) as varchar(20)) || '%' AS 'PNEUMONIA%',
    ROUND(SUM(WHITE_BLOOD_CELLS)/COUNT(WHITE_BLOOD_CELLS),2) as `Avg(White Blood Cells Count(k/uL))`,
    ROUND(SUM(C_REACTIVE_PROTEIN)/COUNT(C_REACTIVE_PROTEIN),2) as `Avg(C-Reactive Protein(mg/dL))`
    FROM tmp
    GROUP BY 1, 2
    '''
    syn3 = pd.read_sql(sql, conn)
    display (syn3)
except Error as e:
    print(e)

,AGE_GROUP,OPIOIDS,PNEUMONIA%,Avg(White Blood Cells Count(k/uL)),Avg(C-Reactive Protein(mg/dL))
0,0,0,1%,10.39,44.08
1,0,1,7%,10.91,54.08
2,1,0,2%,11.12,56.33
3,1,1,10%,11.92,65.74


In [ ]:
try:
    sql='''
    with tmp as (
        SELECT i.*, t.DRUG_NUM,
        CASE WHEN t.DRUG_NUM IS NULL THEN 0 ELSE 1 END AS OPIOIDS
        FROM Important_Statistics as i LEFT JOIN (
          SELECT SUBJECT_ID, COUNT(DRUG) as DRUG_NUM
          FROM drugs
          GROUP BY 1
        )t
        ON i.SUBJECT_ID = t.SUBJECT_ID
    )
    SELECT AGE_GROUP, OPIOIDS, GENDER, CAST(SUM(PNEUMONIA)*100/COUNT(OPIOIDS) as varchar(20)) || '%' AS 'PNEUMONIA%',
    ROUND(SUM(WHITE_BLOOD_CELLS)/COUNT(WHITE_BLOOD_CELLS),2) as `Avg(White Blood Cells Count(k/uL))`,
    ROUND(SUM(C_REACTIVE_PROTEIN)/COUNT(C_REACTIVE_PROTEIN),2) as `Avg(C-Reactive Protein(mg/dL))`
    FROM tmp
    WHERE AGE_GROUP = 1
    GROUP BY 1, 2, 3
    '''
    syn4 = pd.read_sql(sql, conn)
    display (syn4)
except Error as e:
    print(e)

,AGE_GROUP,OPIOIDS,GENDER,PNEUMONIA%,Avg(White Blood Cells Count(k/uL)),Avg(C-Reactive Protein(mg/dL))
0,1,0,F,2%,11.11,54.55
1,1,0,M,2%,11.14,57.93
2,1,1,F,10%,11.96,55.90
3,1,1,M,10%,11.87,75.09
